In [1]:
import _utils as tpx
import pandas as pd
from tiled.client import from_uri
db = from_uri('https://tiled.nsls2.bnl.gov', 'dask')['chx']['raw']

In [2]:
sid = 143200 # example SID (1800 partitions at 1 sec/partition from NSLS-II quantum microscope project)

In [3]:
files = db[sid]['primary']['data']['tpx3_files_raw_filepaths'][0].compute()

file = files[0][5:] # cut off 'file:' from beginning

df = tpx.tpx_to_raw_df(file)
cdf = tpx.raw_df_to_cent_df(df)

print('Raw dataframe: ')
print(df)

print('Centroided dataframe: ')
print(cdf)

cdf = tpx.add_centroid_cols(cdf)
print('Centroided dataframe: ')
print(cdf)

duration = (cdf.iloc[-1]['t_ns'] - cdf.iloc[0]['t_ns'])/1e9
print(f'Partition duration: {duration} sec.')

Raw dataframe: 
          x    y  ToT          t  chip
0       502  452  475   11301032     1
1       405  272  275   11302904     1
2       406  272  350   11302904     1
3       295  135  525   11303039     0
4       358  509  300   11303780     1
...     ...  ...  ...        ...   ...
971254  200   46  550  651295354     3
971255  489  104  450  651296021     0
971256  389   21  300  651296305     0
971257  390   21  300  651296306     0
971258   37  325   75  651299518     2

[971259 rows x 5 columns]
Centroided dataframe: 
                t          xc     yc  ToT_max  ToT_sum  n
0        11301032  502.000000  452.0      475      475  1
1        11302904  405.559998  272.0      350      625  2
2        11303039  295.000000  135.0      525      525  1
3        11303780  358.478271  509.0      300      575  2
4        11303973  412.409088   60.0      325      550  2
...           ...         ...    ...      ...      ... ..
659582  651294809  326.000000  379.0      475      475  1
65

In [4]:
sid = 143201 # new example SID (1800 partitions at 1 sec/partition from NSLS-II quantum microscope project)

In [5]:
files = db[sid]['primary']['data']['tpx3_files_raw_filepaths'][0].compute()
files = [f.replace('file:', '') for f in files]
tpx.convert_tpx3_files(files[0:2])

-> Processing /nsls2/data/chx/legacy/data/2024/02/05/33008fb5-d9f5-4f46-8b5d_00000_000000.tpx3, size: 10.5 MB
Loading /nsls2/data/chx/legacy/data/2024/02/05/33008fb5-d9f5-4f46-8b5d_00000_000000.tpx3 data into dataframe...
Loading /nsls2/data/chx/legacy/data/2024/02/05/33008fb5-d9f5-4f46-8b5d_00000_000000.tpx3 complete. 975938 events found. Saving to: /nsls2/data/chx/legacy/data/2024/02/05/33008fb5-d9f5-4f46-8b5d_00000_000000.h5
Saving /nsls2/data/chx/legacy/data/2024/02/05/33008fb5-d9f5-4f46-8b5d_00000_000000.h5 complete. Beginning clustering...
Clustering /nsls2/data/chx/legacy/data/2024/02/05/33008fb5-d9f5-4f46-8b5d_00000_000000.tpx3 complete. 659537 clusters found. Saving to /nsls2/data/chx/legacy/data/2024/02/05/33008fb5-d9f5-4f46-8b5d_00000_000000_cent.h5
Saving /nsls2/data/chx/legacy/data/2024/02/05/33008fb5-d9f5-4f46-8b5d_00000_000000_cent.h5 complete. Moving onto next file.
-> Processing /nsls2/data/chx/legacy/data/2024/02/05/33008fb5-d9f5-4f46-8b5d_00000_000001.tpx3, size: 10.

In [6]:
df = pd.read_hdf('/nsls2/data/chx/legacy/data/2024/02/05/33008fb5-d9f5-4f46-8b5d_00000_000000.h5')
cdf = pd.read_hdf('/nsls2/data/chx/legacy/data/2024/02/05/33008fb5-d9f5-4f46-8b5d_00000_000000_cent.h5')

print('Raw dataframe: ')
print(df)

print('Centroided dataframe: ')
print(cdf)

cdf = tpx.add_centroid_cols(cdf)
print('Centroided dataframe: ')
print(cdf)

duration = (cdf.iloc[-1]['t_ns'] - cdf.iloc[0]['t_ns'])/1e9
print(f'Partition duration: {duration} sec.')

Raw dataframe: 
          x    y  ToT          t  chip
0       210  132  575   11294477     3
1       106  227  375   11294540     3
2       107  227  150   11294544     3
3       381  346  475   11294882     1
4       268  444  450   11295743     1
...     ...  ...  ...        ...   ...
975933  103  441  600  651291179     2
975934  193  383  450  651294115     2
975935  121  280  400  651294141     2
975936  256  373  400  651294145     1
975937  255  373   50  651294166     2

[975938 rows x 5 columns]
Centroided dataframe: 
                t          xc     yc  ToT_max  ToT_sum  n
0        11294477  210.000000  132.0      575      575  1
1        11294540  106.285713  227.0      375      525  2
2        11294882  381.000000  346.0      475      475  1
3        11295743  268.000000  444.0      450      450  1
4        11298466  420.000000   52.0      450      450  1
...           ...         ...    ...      ...      ... ..
659532  651290601  101.000000  436.0      575      575  1
65

In [7]:
try:
    files = db[sid]['primary']['data']['tpx3_files_raw_filepaths'][0].compute()
    files = [f.replace('file:', '') for f in files]
    tpx.convert_tpx3_files_parallel(files)
    
except Exception as e:
    print(f'Could not finish {sid} due to {e}') 

100%|██████████| 1800/1800 [07:51<00:00,  3.82it/s]


In [9]:
import os

raw_h5_files = [os.path.exists(f.replace('.tpx3', '.h5')) for f in files]
cent_h5_files = [os.path.exists(f.replace('.tpx3', '_cent.h5')) for f in files]

if all(raw_h5_files):
    print('All raw .h5 files exist!')
else:
    print('Missing .h5 files!')
    
if all(cent_h5_files):
    print('All centroided raw .h5 files exist!')
else:
    print('Missing centroided .h5 files!')

All raw .h5 files exist!
All centroided raw .h5 files exist!
